# Demo notebook showing arbok functionalities

This is a simple demo notebook containing the core functionalities of arbok

### Importing qm and arbok module

In [1]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.simulate.credentials import create_credentials
from qm import SimulationConfig

2023-07-13 17:18:30,764 - qm - INFO     - Starting session: abd8ba12-2673-499e-8be9-03a83dd11b05


In [2]:
from arbok.core.quantify_handler import QuantifyHandler
from arbok.core.run_sequence import RunSequence
from arbok.core.sample import Sample

from arbok.samples.sunshine.readout.other_ST_read import OtherStReadout
from arbok.samples.sunshine.initialization.mixed_down_up_init import (
    MixedDownUpInit
)
from arbok.samples.sunshine.configs.rf2v_config import rf2v_config

### Instanciating Arbok modules

Firstly we instanciate `Sequence` objects describing initialization and readout

In [3]:
init = MixedDownUpInit()
readout = OtherStReadout()

Added elements: ['P1', 'J1', 'P2']
Added unit_amp successfully!
Added tPreControl successfully!
Added vHome_P1 successfully!
Added vHome_J1 successfully!
Added vHome_P2 successfully!
Added vDeltaM_P1 successfully!
Added vDeltaM_J1 successfully!
Added vDeltaM_P2 successfully!
Added tPreControlRampMixed successfully!
Added tInitLoadMixed successfully!
Added vInitPreLoadMixed1_P1 successfully!
Added vInitPreLoadMixed1_J1 successfully!
Added vInitPreLoadMixed1_P2 successfully!
Added tInitPreLoad successfully!
Added tInitPreLoadRamp successfully!
Added tControl successfully!
Added tInitLoadRamp successfully!
Added vInitMixed2_P1 successfully!
Added vInitMixed2_J1 successfully!
Added vInitMixed2_P2 successfully!
Added elements: ['P1', 'J1', 'P2']
Added unit_amp successfully!
Added vHome_P1 successfully!
Added vHome_J1 successfully!
Added vHome_P2 successfully!
Added vReference_P1 successfully!
Added vReference_J1 successfully!
Added vReference_P2 successfully!
Added tReadReferenceRamp succes

C:\Users\z5440260\AppData\Local\miniconda3\envs\temp_env\Lib\site-packages\qcodes\instrument\instrument_base.py:143: QCoDeSDeprecationWarning: Parameter read_TIMES on instrument OtherStReadout does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(


TypeError: ParameterBase.__init__() got an unexpected keyword argument 'intial_value'

Next a `Sample` object is created containing the device config. With this we can create an empty `Sequence` that we will populate with other sequences in the following

In [ ]:
readout.tReadRamp

In [ ]:
sunshine = Sample('sunshine', rf2v_config)
qh = RunSequence('CNOT', sunshine)

Note how parameters that have already been defined in 'qh' are skipped.

In [ ]:
readout.read

In [ ]:
qh.add_subsequence(init, verbose = True)
qh.add_subsequence(readout, verbose = True)

Hereby the QCoDeS `Parameter`s describing the respective sequenes are referenced
in the containing `Sequence`. Therefore changes in the subsequences take effect 
in the merged sequence

In [ ]:
print(qh.tInitLoadMixed())
init.tInitLoadMixed(int(1e3/4))
print(qh.tInitLoadMixed())

### Simulating Sequences

The entire sequence can now be simulated. Note that 'tInitLoadMixed' and 'tPreControl' have been reduced for the sake of visibility.

In [ ]:
qh.tPreControl(int(2e3/4))
qh.run_remote_simulation(duration = 4000)

The subsequences can be run as well if we add a sample to their classes

In [ ]:
init.sample = sunshine
init.run_remote_simulation(duration = 4500)

In [ ]:
readout.sample = sunshine
readout.run_remote_simulation(duration = 300)

### Using the `Sequence` in the quantify-core library

In [ ]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl

In [ ]:
dh.set_datadir(dh.default_datadir())

In [ ]:
meas_ctrl = MeasurementControl('meas_ctrl')

In [ ]:
meas_ctrl.settables([qh.tControl, qh.tPreRead])
meas_ctrl.setpoints_grid([
    range(1,300), 
    range(100,300)
])

meas_ctrl.gettables()
dset = meas_ctrl.run(name="Frequency sweep")

In [ ]:
qh.parameters.keys()

In [ ]:
readout.read_read.get_raw()